In [ ]:
import numpy as np
import os
import time

from Airfoils import airfoil as af

from Software.GenuVP import setupGNVP as gnvp
from Plane.planeDefinition import Airplane as Plane

from Database.getresults import Database_2D
from Database import DB3D , BASEGNVP
from Visualization import plotting as aplt

In [ ]:
HOMEDIR = os.getcwd()

In [ ]:
db = Database_2D(HOMEDIR)
airfoils = db.getAirfoils()
polars = db.Data

In [ ]:
WindAngle = 0.
ap = Plane(airfoils, WindAngle)
ap.accessDB(HOMEDIR,DB3D)
# ap.visAirplane()

In [ ]:
cleaning = False
calcGenu = False
calcBatchGenu = True

## Single Run

In [ ]:
ap.angleCASE(WindAngle)
if cleaning == True:
    ap.cleanRes(gnvp.removeResults,[ap.ANGLEDIR,ap.HOMEDIR])
if calcGenu == True:
    genuSetupArgs = [ap.ANGLEDIR,ap.HOMEDIR, BASEGNVP,
                     ap.airMovement,ap.bodies,ap.params,
                     ap.airfoils, polars,"Xfoil"]
    ap.setupSolver(gnvp.makeInput,genuSetupArgs)
    ap.runSolver(gnvp.runGNVP,[HOMEDIR,ap.ANGLEDIR])
# ap.makePolars(gnvp.getData,"GNVP")

## Batch Run

In [ ]:
AoAmax = 15
AoAmin = -6
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)

In [ ]:
ap.batchangles(angles)
if calcBatchGenu == True:
    genuBatchArgs = [ap.CASEDIR,ap.HOMEDIR, BASEGNVP,
                     ap.airMovement,ap.bodies,ap.params,
                     ap.airfoils, polars,"Xfoil",angles]
    ap.runSolver(gnvp.batchRun,genuBatchArgs)
    